In [2]:
!pip install tensorflow

^C


     ---------------------------------------- 0.0/61.0 kB ? eta -:--:--
     ------ --------------------------------- 10.2/61.0 kB ? eta -:--:--
     --------------------------------- ------ 51.2/61.0 kB 1.3 MB/s eta 0:00:01
     -------------------------------------- 61.0/61.0 kB 651.3 kB/s eta 0:00:00
  Using cached wheel-0.43.0-py3-none-any.whl.metadata (2.2 kB)
     ---------------------------------------- 0.0/48.7 kB ? eta -:--:--
     ---------------------------------------- 48.7/48.7 kB 1.2 MB/s eta 0:00:00
   ---------------------------------------- 0.0/385.2 MB ? eta -:--:--
   ---------------------------------------- 0.1/385.2 MB 2.6 MB/s eta 0:02:31
   ---------------------------------------- 0.1/385.2 MB 3.2 MB/s eta 0:02:01
   ---------------------------------------- 0.2/385.2 MB 2.5 MB/s eta 0:02:35
   ---------------------------------------- 0.3/385.2 MB 1.7 MB/s eta 0:03:43
   ---------------------------------------- 0.4/385.2 MB 1.9 MB/s eta 0:03:28
   ----------------

  You can safely remove it manually.
  You can safely remove it manually.

[notice] A new release of pip is available: 24.1.1 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import cv2
import numpy as np
import time
import os
import HandTrackingModule as htm
import matplotlib.pyplot as plt
import tensorflow as tf
from keras import datasets, layers, models
import math
import wmi
from comtypes import CLSCTX_ALL
from pycaw.pycaw import AudioUtilities, IAudioEndpointVolume

In [5]:
###########################
brushThickness = 30
eraserThickness = 100
MaxBrightness = 100
MinBrightness = 0
devices = AudioUtilities.GetSpeakers()
interface = devices.Activate(
    IAudioEndpointVolume._iid_, CLSCTX_ALL, None)
volume = interface.QueryInterface(IAudioEndpointVolume)
volRange = volume.GetVolumeRange()
minVol = volRange[0]
maxVol = volRange[1]
vol = 0
volBar = 400
volPer = 0
###########################
folderPath = "headerImages"
myList = os.listdir(folderPath)
print(myList)
overlayList = []
for imPath in myList:
    image = cv2.imread(f'{folderPath}/{imPath}')
    overlayList.append(image)
print(len(overlayList))
header = overlayList[0]
c = wmi.WMI(namespace='wmi')
drawColor = False


cap = cv2.VideoCapture(0)
cap.set(3, 1280)
cap.set(4, 720)

detector = htm.handDetector(detectionCon=0.85)
xp, yp = 0, 0
imgCanvas = np.zeros((720, 1280, 3), np.uint8)
brightnessControl = False
volControl = False

while True:
    success, img = cap.read()
    img = cv2.flip(img, 1)
    img = detector.findHands(img)
    lmList = detector.findPosition(img, draw=False)   
    if len(lmList) != 0:
        print(lmList[8])
        x0, y0 = lmList[4][1:]
        x1, y1 = lmList[8][1:]  
        x2, y2 = lmList[12][1:]
        cx, cy = (x1+x0)//2, (y1+y0)//2

        fingers = detector.fingersUp()
        #print(fingers)
        if volControl:
            if fingers[1] and fingers[2] == False:
                cv2.circle(img, (x0,y0), 15, (0, 0, 0), cv2.FILLED)
                cv2.circle(img, (x1, y1), 15, (0, 0 ,0), cv2.FILLED)
                cv2.line(img, (x0,y0),(x1,y1), (0, 0, 0), 3)
                cv2.circle(img, (cx, cy), 15, (0, 0, 255), cv2.FILLED)
                length = math.hypot(x1-x0, y1-y0)
                vol = np.interp(length,[50,250],[minVol, maxVol])
                volBar = np.interp(length,[50, 250], [600, 250])
                volPer = np.interp(length, [50, 250], [0, 100])
                #print(int(length), vol)
                volume.SetMasterVolumeLevel(vol, None)
                color = (0, int(255 - (brightness / MaxBrightness) * 255), int((brightness / MaxBrightness) * 255))  
                if length<50:
                    cv2.circle(img, (cx, cy), 15, (0, 255, 0), cv2.FILLED)
                cv2.rectangle(img, (50,250), (85,600), color, 3)
                cv2.rectangle(img, (50,int(volBar)), (85,600), color, cv2.FILLED)
                cv2.putText(img, f'{int(volPer)} %',(45,220), cv2.FONT_HERSHEY_COMPLEX,
                    1,color, 3)


        if brightnessControl:
                if fingers[1] and fingers[2] == False:
                    cv2.circle(img, (x0, y0), 15, (0, 0, 0), cv2.FILLED)
                    cv2.circle(img, (x1, y1), 15, (0, 0, 0), cv2.FILLED)
                    cv2.line(img, (x0,y0),(x1,y1), (0, 0, 0), 3)
                    cv2.circle(img, (cx, cy), 15, (0, 0, 255), cv2.FILLED)
                    Length = math.hypot(x1-x0, y1-y0)
                    if Length<50:
                        cv2.circle(img, (cx, cy), 15, (0, 255, 0), cv2.FILLED)
                    length = np.interp(Length, (50,250), (MinBrightness, MaxBrightness))
                    brightnessBar = np.interp(Length, (50, 250), (600, 250))
                    brightness = int(length) 
                    color = (0, int(255 - (brightness / MaxBrightness) * 255), int((brightness / MaxBrightness) * 255))  
                    if length <= MaxBrightness:
                        color = (0, int(255 - (brightness / MaxBrightness) * 255), int((brightness / MaxBrightness) * 255))
                        cv2.rectangle(img, (50,250), (85,600), color, 3)
                        cv2.rectangle(img, (50,int(brightnessBar)), (85,600), color, cv2.FILLED)
                        cv2.putText(img, f'{int(brightness)} %',(45,220), cv2.FONT_HERSHEY_COMPLEX,
                            1,(0, int(255 - (brightness / MaxBrightness) * 255), int((brightness / MaxBrightness) * 255)), 3)
                    c.WmiMonitorBrightnessMethods()[0].WmiSetBrightness(brightness, 0)
                    

        if fingers[1] and fingers[2]:
            xp, yp = 0, 0
            cv2.rectangle(img, (x1, y1 - 25), (x2, y2 + 25), drawColor, cv2.FILLED)
            #print("Selection Mode")
            if y1 < 125:
                if 690 < x1 < 730:
                    brightnessControl = False
                    volControl = False
                    header = overlayList[1]
                    drawColor = (0, 0, 255)
                elif 790 < x1 < 830:
                    brightnessControl = False
                    volControl = False
                    header = overlayList[2]
                    drawColor = (220, 35, 35)
                elif 890 < x1 < 930:
                    brightnessControl = False
                    volControl = False
                    header = overlayList[3]
                    drawColor = (0, 255, 0)
                elif 990 < x1 < 1030:
                    brightnessControl = False
                    volControl = False
                    header = overlayList[4]
                    drawColor = (255, 255, 255)
                elif 1090 < x1 < 1165:
                    brightnessControl = False
                    volControl = False
                    header = overlayList[5]
                    drawColor = (0, 0, 0)
                elif 50 < x1 < 100:
                    header = overlayList[7]
                    volControl = False
                    drawColor = False
                    brightnessControl = True
                elif 150 < x1 < 250:
                    header = overlayList[6]
                    drawColor = False
                    brightnessControl = False
                    volControl = True
                elif 330 < x1 < 460:
                    header = overlayList[8]
                    drawColor = False
                    brightnessControl = False
                    volControl = False
            

        if fingers[1] and fingers[2] == False:
            cv2.circle(img, (x1, y1), 15, drawColor, cv2.FILLED)
            #print("Drawing Mode")
            if xp == 0 and yp == 0:
                xp, yp = x1, y1

            if drawColor == (0,0,0):
                cv2.line(img, (xp,yp), (x1,y1), drawColor, eraserThickness)
                cv2.line(imgCanvas, (xp, yp), (x1, y1), drawColor, eraserThickness)
            else:    
                cv2.line(img, (xp,yp), (x1,y1), drawColor, brushThickness)
                cv2.line(imgCanvas, (xp, yp), (x1, y1), drawColor, brushThickness)
            xp, yp = x1, y1

    imgGray = cv2.cvtColor(imgCanvas, cv2.COLOR_BGR2GRAY)
    _, imgInv = cv2.threshold(imgGray, 50, 255, cv2.THRESH_BINARY_INV) 
    imgInv = cv2.cvtColor(imgInv, cv2.COLOR_GRAY2BGR) 
    img = cv2.bitwise_and(img,imgInv)
    img = cv2.bitwise_or(img, imgCanvas)

    img[0:125, 0:1280] = header
    #img = cv2.addWeighted(img,0.5,imgCanvas,0.5)
    cv2.imshow("Image", img)
    cv2.imshow("Canvas", imgCanvas)
    cv2.waitKey(1)
    if cv2.waitKey(1) and 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

FileNotFoundError: [WinError 3] The system cannot find the path specified: 'headerImages'